In [19]:
import pandas as pd
import numpy as np

df=pd.read_csv(r'C:\Users\ali.karimi\Documents\JupyterProjects\playground-series-s6e1\train.csv')
dft=pd.read_csv(r'C:\Users\ali.karimi\Documents\JupyterProjects\playground-series-s6e1\test.csv')


print (df.shape, dft.shape)
df.head()

(630000, 13) (270000, 12)


,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


In [20]:

df['internet_access']=df['internet_access'].replace({'yes':1,'no' :0})
dft['internet_access']=dft['internet_access'].replace({'yes':1,'no' :0})

df['sleep_quality']=df['sleep_quality'].replace({'poor':0,'average' :1,'good':2})
dft['sleep_quality']=dft['sleep_quality'].replace({'poor':0,'average' :1,'good':2})

df['facility_rating']=df['facility_rating'].replace({'low':0,'medium' :1,'high':2})
dft['facility_rating']=dft['facility_rating'].replace({'low':0,'medium' :1,'high':2})

df['exam_difficulty']=df['exam_difficulty'].replace({'easy':0,'moderate' :1,'hard':2})
dft['exam_difficulty']=dft['exam_difficulty'].replace({'easy':0,'moderate' :1,'hard':2})

df['course']=df['course'].replace({'b.tech':0,'b.sc' :1,'b.com':2,'bca':3, 'bba':4, 'ba':5, 'diploma':6})
dft['course']=df['course'].replace({'b.tech':0,'b.sc' :1,'b.com':2,'bca':3, 'bba':4, 'ba':5,'diploma':6 })


                                                    
df['gender']=df['gender'].replace({'male':0,'female' :1,'other':2})
dft['gender']=dft['gender'].replace({'male':0,'female' :1,'other':2})

df['study_method']=df['study_method'].replace({'coaching':0,'self-study' :1,'mixed':2,'group study':3, 'online videos':4})
dft['study_method']=dft['study_method'].replace({'coaching':0,'self-study' :1,'mixed':2,'group study':3, 'online videos':4})


C:\Users\ali.karimi\AppData\Local\Temp\ipykernel_21044\2618125061.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['internet_access']=df['internet_access'].replace({'yes':1,'no' :0})
C:\Users\ali.karimi\AppData\Local\Temp\ipykernel_21044\2618125061.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dft['internet_access']=dft['internet_access'].replace({'yes':1,'no' :0})
C:\Users\ali.karimi\AppData\Local\Temp\ipykernel_21044\2618125061.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed i

In [8]:
# df_tree=df.drop(columns=['id','gender','course','exam_difficulty', 'internet_access','age','study_method'], axis=1)
# dft_tree=dft.drop(columns=['id','gender','course','exam_difficulty', 'internet_access','age','study_method'], axis=1)


In [66]:
df_tree.head()

,study_hours,class_attendance,sleep_hours,sleep_quality,facility_rating,exam_score
0,7.91,98.8,4.9,1,0,78.3
1,4.95,94.8,4.7,0,1,46.7
2,4.68,92.6,5.8,0,2,99.0
3,2.00,49.5,8.3,1,2,63.9
4,7.65,86.9,9.6,2,2,100.0


In [100]:
y=df['exam_score']
X=df.iloc[:,1:-1].drop(columns=['exam_difficulty','course','gender','internet_access','age'], axis=1)
X.head()

,study_hours,class_attendance,sleep_hours,sleep_quality,study_method,facility_rating
0,7.91,98.8,4.9,1,4,0
1,4.95,94.8,4.7,0,1,1
2,4.68,92.6,5.8,0,0,2
3,2.00,49.5,8.3,1,3,2
4,7.65,86.9,9.6,2,1,2


In [99]:
# اینو بعد اجرای مدل ها اجرا کردم. برای این که ببینم کدوم فیچرها رو حذف کنم

features = X.columns.tolist()
xgb_importance = (
    pd.DataFrame(
        xgb.get_booster().get_score(importance_type="gain").items(),
        columns=["feature", "xgb_gain"]
    )
)

xgb_importance["xgb_gain"] /= xgb_importance["xgb_gain"].sum()


lgb_importance = pd.DataFrame({
    "feature": X.columns,
    "lgb_gain": lgb.booster_.feature_importance(importance_type="gain")
})

lgb_importance["lgb_gain"] /= lgb_importance["lgb_gain"].sum()

cat_importance = cat.get_feature_importance(
    type="PredictionValuesChange"
)

cat_importance = pd.DataFrame({
    "feature": X.columns,
    "cat_importance": cat_importance
})

cat_importance["cat_importance"] /= cat_importance["cat_importance"].sum()

importance_df = (
    xgb_importance
    .merge(lgb_importance, on="feature", how="outer")
    .merge(cat_importance, on="feature", how="outer")
).fillna(0)

importance_df["mean_importance"] = importance_df[
    ["xgb_gain", "lgb_gain", 'cat_importance']
].mean(axis=1)

importance_df = importance_df.sort_values(
    "mean_importance", ascending=False
)

importance_df.head(10)


,feature,xgb_gain,lgb_gain,cat_importance,mean_importance
6,study_hours,0.533966,0.723897,0.515473,0.591112
1,class_attendance,0.088633,0.122003,0.208068,0.139568
5,sleep_quality,0.159620,0.051028,0.088067,0.099572
7,study_method,0.083406,0.043318,0.086339,0.071021
2,facility_rating,0.105788,0.033738,0.058352,0.065959
4,sleep_hours,0.023340,0.024535,0.041531,0.029802
0,age,0.002785,0.001288,0.001814,0.001962
3,internet_access,0.002460,0.000193,0.000356,0.001003


In [101]:
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from xgboost.callback import EarlyStopping


def objective(trial):

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 1000, 9000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.09, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.75, 0.95),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.75, 0.95),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.5, 3.0),
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "tree_method": "hist",
        "random_state": 42,
        'early_stopping_rounds': 50
    }

    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    oof_pred = np.zeros(len(X))

    for tr_idx, val_idx in kf.split(X, y):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        model = XGBRegressor(**params,)

        model.fit(
            X_tr,
            y_tr,
            eval_set=[(X_val, y_val)],
            verbose=False,
        )

        oof_pred[val_idx] = model.predict(X_val)

    rmse = np.sqrt(mean_squared_error(y, oof_pred))
    return rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

print("Best XGB RMSE:", study.best_value)
print("Best XGB params:", study.best_params)


[I 2026-01-31 14:37:42,271] A new study created in memory with name: no-name-81856aea-0b15-42a3-a7ef-9101482853c5
[I 2026-01-31 14:39:03,553] Trial 0 finished with value: 8.772558294024371 and parameters: {'n_estimators': 6011, 'learning_rate': 0.022833190196389182, 'max_depth': 10, 'min_child_weight': 2, 'subsample': 0.7830489148392548, 'colsample_bytree': 0.8815798412646968, 'reg_alpha': 0.1336616595743445, 'reg_lambda': 2.443989839546181}. Best is trial 0 with value: 8.772558294024371.
[I 2026-01-31 14:47:09,277] Trial 1 finished with value: 8.749028447170893 and parameters: {'n_estimators': 8735, 'learning_rate': 0.016315268407105184, 'max_depth': 5, 'min_child_weight': 10, 'subsample': 0.8858730510818302, 'colsample_bytree': 0.8824800571881539, 'reg_alpha': 0.5368010565416671, 'reg_lambda': 1.3267467572639715}. Best is trial 1 with value: 8.749028447170893.
[I 2026-01-31 14:47:54,502] Trial 2 finished with value: 8.761624342301612 and parameters: {'n_estimators': 1838, 'learning_r

Best XGB RMSE: 8.748978311914659
Best XGB params: {'n_estimators': 7497, 'learning_rate': 0.01620269681716931, 'max_depth': 6, 'min_child_weight': 9, 'subsample': 0.8469353140464527, 'colsample_bytree': 0.942610539792036, 'reg_alpha': 0.8080329736367569, 'reg_lambda': 1.4663816823311906}


In [102]:
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor


def objective(trial):

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 1000, 7000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.09, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.75, 0.95),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.75, 0.95),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.5, 3.0),
        "random_state": 42,
        'early_stopping_rounds': 50,
        'verbose':0
    }

    kf = KFold(n_splits=3, shuffle=True, random_state=1)
    oof_pred = np.zeros(len(X))

    for tr_idx, val_idx in kf.split(X, y):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        model = LGBMRegressor(**params,)

        model.fit(
            X_tr,
            y_tr,
            eval_set=[(X_val, y_val)],
            eval_metric="rmse"
        )

        oof_pred[val_idx] = model.predict(X_val)

    rmse = np.sqrt(mean_squared_error(y, oof_pred))
    return rmse

study_lgb = optuna.create_study(direction="minimize")
study_lgb.optimize(objective, n_trials=30)

print("Best LGBM RMSE:", study_lgb.best_value)
print("Best LGBM params:", study_lgb.best_params)



[I 2026-01-31 17:03:31,802] A new study created in memory with name: no-name-56fb6e98-9a3f-4c85-8dd1-73d654c3fd84


[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50


[I 2026-01-31 17:04:51,322] Trial 0 finished with value: 8.774229765288744 and parameters: {'n_estimators': 1489, 'learning_rate': 0.020904871243015124, 'min_child_weight': 5, 'subsample': 0.8116822611369183, 'colsample_bytree': 0.7679678725113529, 'reg_alpha': 0.9505206901835241, 'reg_lambda': 2.526652408005133}. Best is trial 0 with value: 8.774229765288744.


[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50


[I 2026-01-31 17:06:32,435] Trial 1 finished with value: 8.78699354463886 and parameters: {'n_estimators': 1609, 'learning_rate': 0.011001689623534278, 'min_child_weight': 7, 'subsample': 0.9387910752326444, 'colsample_bytree': 0.8068155973066566, 'reg_alpha': 0.9051915190199507, 'reg_lambda': 2.497528860379078}. Best is trial 0 with value: 8.774229765288744.


...

[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50


[I 2026-01-31 18:20:47,060] Trial 28 finished with value: 8.760881857645963 and parameters: {'n_estimators': 6374, 'learning_rate': 0.03146237050950133, 'min_child_weight': 3, 'subsample': 0.7863451004713417, 'colsample_bytree': 0.844773632588446, 'reg_alpha': 0.58868455840804, 'reg_lambda': 2.011286311152127}. Best is trial 8 with value: 8.760629492295072.


[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50


[I 2026-01-31 18:23:36,152] Trial 29 finished with value: 8.763287483882532 and parameters: {'n_estimators': 5770, 'learning_rate': 0.017097839452609415, 'min_child_weight': 4, 'subsample': 0.8093965880037783, 'colsample_bytree': 0.9316912651117754, 'reg_alpha': 0.7800409109939541, 'reg_lambda': 2.350953314458276}. Best is trial 8 with value: 8.760629492295072.


Best LGBM RMSE: 8.760629492295072
Best LGBM params: {'n_estimators': 5116, 'learning_rate': 0.018299595510518348, 'min_child_weight': 3, 'subsample': 0.8160499738882326, 'colsample_bytree': 0.815002258798347, 'reg_alpha': 0.06924814898223275, 'reg_lambda': 2.993185202846954}


In [78]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np
from catboost import CatBoostRegressor


def objective(trial):

    params = {
        "iterations": trial.suggest_int("iterations", 1000, 7000),
        "depth": trial.suggest_int('depth',3,10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.09, log=True),
        "random_state": 142,
        'early_stopping_rounds': 50,
        'loss_function':'RMSE',
        'verbose':0
    }

    kf = KFold(n_splits=3, shuffle=True, random_state=141)
    oof_pred = np.zeros(len(X))

    for tr_idx, val_idx in kf.split(X, y):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        model = CatBoostRegressor(**params,)

        model.fit(
            X_tr,
            y_tr,
            eval_set=[(X_val, y_val)],
        )

        oof_pred[val_idx] = model.predict(X_val)

    rmse = np.sqrt(mean_squared_error(y, oof_pred))
    return rmse

study_cat = optuna.create_study(direction="minimize")
study_cat.optimize(objective, n_trials=30)

print("Best CAT RMSE:", study_cat.best_value)
print("Best CAT params:", study_cat.best_params)


[I 2026-01-28 14:25:59,700] A new study created in memory with name: no-name-7e0b1787-d166-40a3-87c2-7796ce876d5f
[I 2026-01-28 14:33:57,035] Trial 0 finished with value: 8.799394377171906 and parameters: {'n_estimators': 6358, 'iterations': 3, 'learning_rate': 0.015729597557600228}. Best is trial 0 with value: 8.799394377171906.
[I 2026-01-28 14:44:13,889] Trial 1 finished with value: 8.772795329458432 and parameters: {'n_estimators': 5762, 'iterations': 6, 'learning_rate': 0.010645883469435376}. Best is trial 1 with value: 8.772795329458432.
[I 2026-01-28 14:53:17,902] Trial 2 finished with value: 8.753945219424065 and parameters: {'n_estimators': 5938, 'iterations': 8, 'learning_rate': 0.02997732170068921}. Best is trial 2 with value: 8.753945219424065.
[I 2026-01-28 15:04:06,437] Trial 3 finished with value: 8.767694824857838 and parameters: {'n_estimators': 4444, 'iterations': 8, 'learning_rate': 0.011102830737200382}. Best is trial 2 with value: 8.753945219424065.
[I 2026-01-28 1

Best CAT RMSE: 8.74748003150613
Best CAT params: {'n_estimators': 5832, 'iterations': 6, 'learning_rate': 0.06569495662219477}


In [83]:
from sklearn.ensemble import RandomForestRegressor


# =========================
# PARAMS
# =========================
rf_param = {
    "n_estimators": 300,
    "max_depth": 20,
    "min_samples_split": 6,
    "min_samples_leaf": 4,
    "max_features": "sqrt",
    "n_jobs": -1,
    "random_state": 42
}

# =========================
# INIT
# =========================
oof_rf_pred = np.zeros(len(X))


n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=4)

# =========================
# CV LOOP
# =========================
for fold, (tr_idx, val_idx) in enumerate(kf.split(X, y)):

    print(f"Fold {fold+1}")

    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    rf = RandomForestRegressor(**rf_param)

    rf.fit(X_tr, y_tr)

    # ---- OOF ----
    oof_rf_pred[val_idx] = rf.predict(X_val)

    # ---- TEST (میانگین فولدها) ----


# =========================
# OOF SCORE
# =========================
oof_rf_rmse = np.sqrt(mean_squared_error(y, oof_rf_pred))
print("OOF RF RMSE:", oof_rf_rmse)


Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
OOF RF RMSE: 8.890526937997253


In [84]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

n_folds = 5

kf = KFold(n_splits=n_folds, 
           shuffle=True, random_state=172
          )
oof_xgb_pred = np.zeros(len(X))
oof_lgb_pred = np.zeros(len(X))
oof_cat_pred = np.zeros(len(X))
oof_rf_pred = np.zeros(len(X))



for fold, (tr_idx, val_idx) in enumerate(kf.split(X, y)):

    print(f"Fold {fold+1}")

    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    xgb = XGBRegressor(**study.best_params, objective= "reg:squarederror",eval_metric= "rmse",tree_method= "hist", random_state= 42, early_stopping_rounds= 50)
    lgb=LGBMRegressor(**study_lgb.best_params,random_state= 42,verbose=0)
    cat=CatBoostRegressor(iterations=5832, depth= 6, learning_rate= 0.06569495662219477, verbose=0, random_state =42, early_stopping_rounds= 50, loss_function='RMSE')
    rf=RandomForestRegressor(**rf_param)

    xgb.fit(X_tr, y_tr,eval_set=[(X_val, y_val)],verbose=False)
    lgb.fit(X_tr, y_tr,eval_set=[(X_val, y_val)], eval_metric="rmse")
    cat.fit(X_tr, y_tr,eval_set=[(X_val, y_val)])
    rf.fit(X_tr,y_tr)

    # ---- OOF ----
    oof_xgb_pred[val_idx] = xgb.predict(X_val)
    oof_lgb_pred[val_idx] = lgb.predict(X_val)
    oof_cat_pred[val_idx] = cat.predict(X_val)
    oof_rf_pred[val_idx] = rf.predict(X_val)

    # ---- TEST (میانگین فولدها) ----


# =========================
# OOF SCORE
# =========================
oof_xgb_rmse = np.sqrt(mean_squared_error(y, oof_xgb_pred))
print("OOF XGB RMSE:", oof_xgb_rmse)
oof_lgb_rmse = np.sqrt(mean_squared_error(y, oof_lgb_pred))
print("OOF LGB RMSE:", oof_lgb_rmse)
oof_cat_rmse = np.sqrt(mean_squared_error(y, oof_cat_pred))
print("OOF CAT RMSE:", oof_cat_rmse)
oof_rf_rmse = np.sqrt(mean_squared_error(y, oof_rf_pred))
print("OOF RF RMSE:", oof_rf_rmse)


X_stack = np.vstack([
    oof_xgb_pred,
    oof_lgb_pred,
    oof_cat_pred,
    oof_rf_pred 
]).T

oof_ridge = np.zeros(len(y))
kf = KFold(n_splits=5, shuffle=True, random_state=172)

for tr_idx, val_idx in kf.split(X_stack):
    ridge = Ridge(alpha=1.5)
    ridge.fit(X_stack[tr_idx], y.iloc[tr_idx])
    oof_ridge[val_idx] = ridge.predict(X_stack[val_idx])

rmse = np.sqrt(mean_squared_error(y, oof_ridge))

    
    
print("Ridge OOF RMSE (realistic):", rmse)
print (ridge.coef_)

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
OOF XGB RMSE: 8.744339472152095
OOF LGB RMSE: 8.752233378446407
OOF CAT RMSE: 8.746901685161886
OOF RF RMSE: 8.892323417155835
Ridge OOF RMSE (realistic): 8.734536968023823
[ 0.39598222  0.25406976  0.37152872 -0.02051772]


In [85]:
# from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge



xgb = XGBRegressor(**study.best_params, objective= "reg:squarederror",eval_metric= "rmse",tree_method= "hist", random_state= 42, early_stopping_rounds= 50)
xgb.fit(X, y,eval_set=[(X_val, y_val)],verbose=False)
y_xgb2=xgb.predict(X)
print('XGB RMSE:',np.sqrt(mean_squared_error(y, y_xgb2)))


lgb= LGBMRegressor(**study_lgb.best_params,random_state= 42,verbose=0)
lgb.fit(X,y,eval_set=[(X_val, y_val)],eval_metric="rmse")
y_lgb2=lgb.predict(X)
print('LGBM RMSE:',np.sqrt(mean_squared_error(y, y_lgb2)))

cat = CatBoostRegressor(iterations=4381, depth= 5, learning_rate= 0.06309859776890298, verbose=0, random_state =42, early_stopping_rounds= 50, loss_function='RMSE')
cat.fit(X,y,eval_set=[(X_val, y_val)])
y_cat2=cat.predict(X)
print('CAT RMSE:',np.sqrt(mean_squared_error(y, y_cat2)))

rf = RandomForestRegressor(**rf_param)
rf.fit(X,y)
y_rf2=rf.predict(X)
print('RF RMSE:',np.sqrt(mean_squared_error(y, y_rf2)))



X_stack2 = np.vstack([
    y_xgb2,
    y_lgb2,
    y_cat2,
    y_rf2 
]).T

y_ridge2 = ridge.predict(X_stack)

rmse = np.sqrt(mean_squared_error(y, y_ridge2))

    
    
print("Ridge OOF RMSE (just on train):", rmse)
print (ridge.coef_)




XGB RMSE: 8.564258366417183
LGBM RMSE: 8.557838722688365
CAT RMSE: 8.654092538522614
RF RMSE: 7.2020440411141555
Ridge OOF RMSE (just on train): 8.7343763038006
[ 0.39598222  0.25406976  0.37152872 -0.02051772]


In [89]:
test = pd.read_csv(r"C:\Users\ali.karimi\Documents\JupyterProjects\playground-series-s6e1\sample_submission.csv")
Xt=dft.iloc[:,1:].drop(columns=['exam_difficulty','course','gender','internet_access','age'], axis=1)

yt_xgb=xgb.predict(Xt)
yt_lgb=lgb.predict(Xt)
yt_cat=cat.predict(Xt)
yt_rf=rf.predict(Xt)
Xt_stack = np.vstack([yt_xgb, yt_lgb ,yt_cat,yt_rf]).T
yt = ridge.predict(Xt_stack)

# Prepare submission
submission = pd.DataFrame({
    "id": test['id'],          # از sample_submission بخون
    "exam_score": yt  # probability برای AUC
})

submission.to_csv("submission.csv", index=False)
print("Submission file created!")

Submission file created!


In [49]:
ridge.coef_
#در ادامه سودو هست که به درد نمی خوره

array([ 0.12540244, -0.90763175, -1.36231386,  3.21338232])

In [92]:
N_SPLITS = 5
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

oof_pred = np.zeros(len(X))
test_pred = np.zeros(len(Xt))

for fold, (tr_idx, val_idx) in enumerate(kf.split(X), 1):

    print(f"Fold {fold}")

    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    # =====================
    # a) MODEL 1 (real labels only)
    # =====================
    model_1 = XGBRegressor(**study.best_params, objective= "reg:squarederror",eval_metric= "rmse",tree_method= "hist", random_state= 42, early_stopping_rounds= 50)

    model_1.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    val_pred_1 = model_1.predict(X_val)
    test_pred_1 = model_1.predict(Xt)

    # =====================
    # b) BUILD PSEUDO DATASET
    # =====================
    X_pseudo = pd.concat([X_tr, X_val, Xt], axis=0)
    y_pseudo = np.concatenate([
        y_tr.values,
        val_pred_1,
        test_pred_1
    ])

    # =====================
    # c) MODEL 2 (real + pseudo)
    # =====================
    model_2 = XGBRegressor(**study.best_params, objective= "reg:squarederror",eval_metric= "rmse",tree_method= "hist", random_state= 42, early_stopping_rounds= 50)

    model_2.fit(
        X_pseudo,
        y_pseudo,
        verbose=False
    )

    # =====================
    # d) PREDICT
    # =====================
    oof_pred[val_idx] = model_2.predict(X_val)
    test_pred += model_2.predict(Xt) / N_SPLITS


rmse = np.sqrt(mean_squared_error(y, oof_pred))
print("OOF RMSE with pseudo:", rmse)

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
OOF RMSE with pseudo: 8.748397829713952


In [95]:
N_SPLITS = 5
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

oof_pred = np.zeros(len(X))
test_pred_lgbm = np.zeros(len(Xt))

for fold, (tr_idx, val_idx) in enumerate(kf.split(X), 1):

    print(f"Fold {fold}")

    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    # =====================
    # a) MODEL 1 (real labels only)
    # =====================
    model_lgb_1 = LGBMRegressor(**study_lgb.best_params,random_state= 42,verbose=0)

    model_lgb_1.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
    )

    val_pred_1 = model_lgb_1.predict(X_val)
    test_pred_1 = model_lgb_1.predict(Xt)

    # =====================
    # b) BUILD PSEUDO DATASET
    # =====================
    X_pseudo = pd.concat([X_tr, X_val, Xt], axis=0)
    y_pseudo = np.concatenate([
        y_tr.values,
        val_pred_1,
        test_pred_1
    ])

    # =====================
    # c) MODEL 2 (real + pseudo)
    # =====================
    model_lgb_2 = LGBMRegressor(**study_lgb.best_params,random_state= 42,verbose=0)

    model_lgb_2.fit(
        X_pseudo,
        y_pseudo,
        eval_set=[(X_val, y_val)]
    )

    # =====================
    # d) PREDICT
    # =====================
    oof_pred[val_idx] = model_lgb_2.predict(X_val)
    test_pred_lgbm += model_lgb_2.predict(Xt) / N_SPLITS


rmse = np.sqrt(mean_squared_error(y, oof_pred))
print("OOF LGBM RMSE with pseudo:", rmse)

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
OOF LGBM RMSE with pseudo: 8.752596395601978


In [96]:
N_SPLITS = 5
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

oof_pred = np.zeros(len(X))
test_pred_cat = np.zeros(len(Xt))

for fold, (tr_idx, val_idx) in enumerate(kf.split(X), 1):

    print(f"Fold {fold}")

    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    # =====================
    # a) MODEL 1 (real labels only)
    # =====================
    model_cat_1 = CatBoostRegressor(iterations=5832, depth= 6, learning_rate= 0.06569495662219477, verbose=0, random_state =42, early_stopping_rounds= 50, loss_function='RMSE')

    model_cat_1.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
    )

    val_pred_1 = model_cat_1.predict(X_val)
    test_pred_1 = model_cat_1.predict(Xt)

    # =====================
    # b) BUILD PSEUDO DATASET
    # =====================
    X_pseudo = pd.concat([X_tr, X_val, Xt], axis=0)
    y_pseudo = np.concatenate([
        y_tr.values,
        val_pred_1,
        test_pred_1
    ])

    # =====================
    # c) MODEL 2 (real + pseudo)
    # =====================
    model_cat_2 = CatBoostRegressor(iterations=5832, depth= 6, learning_rate= 0.06569495662219477, verbose=0, random_state =42, early_stopping_rounds= 50, loss_function='RMSE')


    model_cat_2.fit(
        X_pseudo,
        y_pseudo,
        eval_set=[(X_val, y_val)]
    )

    # =====================
    # d) PREDICT
    # =====================
    oof_pred[val_idx] = model_cat_2.predict(X_val)
    test_pred_cat += model_cat_2.predict(Xt) / N_SPLITS


rmse = np.sqrt(mean_squared_error(y, oof_pred))
print("OOF LGBM RMSE with pseudo:", rmse)

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
OOF LGBM RMSE with pseudo: 8.747164108583062
